<a href="https://colab.research.google.com/github/ManuelRix-11/FlowerRecognition/blob/main/ProgettoFia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import delle Librerie 📓

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

## Percorsi delle cartelle e creazione dei sotto set 📁

In [ ]:
base_dir = '/content/drive/MyDrive/working/Images'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

def split_data(base_dir, working_dir, train_ratio=0.7, val_ratio=0.2):
    for folder in ['train', 'val', 'test']:
        folder_path = os.path.join(working_dir, folder)
        if os.path.exists(folder_path):
            shutil.rmtree(folder_path)
        os.makedirs(folder_path)

    for category in os.listdir(base_dir):
        category_path = os.path.join(base_dir, category)
        if os.path.isdir(category_path):
            images = os.listdir(category_path)
            np.random.shuffle(images)

            train_split = int(len(images) * train_ratio)
            val_split = int(len(images) * (train_ratio + val_ratio))

            train_images = images[:train_split]
            val_images = images[train_split:val_split]
            test_images = images[val_split:]

            for image_set, folder in zip([train_images, val_images, test_images], ['train', 'val', 'test']):
                target_dir = os.path.join(working_dir, folder, category)
                os.makedirs(target_dir, exist_ok=True)
                for image in image_set:
                    shutil.copy(os.path.join(category_path, image), os.path.join(target_dir, image))

base_dir = '/content/drive/MyDrive/flowers'
working_dir = '/content/drive/MyDrive/working/Images'

split_data(base_dir, working_dir)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/flowers'

## Generatori di immagini 🖼



In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

## Costruzione del Modello 🏗

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Dropout(0.5),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.5),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.5),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.5),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Addestramento 🏃🏻‍♂️

In [ ]:
history = model.fit(
    train_generator,
    epochs=25,
    validation_data=val_generator
)

## Valutazione ✅

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")

## Predizione

In [ ]:
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

## Output delle predizioni

In [ ]:
def display_predictions(generator, predictions, true_classes, class_indices, num_samples=30):
    class_labels = list(class_indices.keys())
    indices = np.arange(len(predictions))
    np.random.shuffle(indices)
    sampled_indices = indices[:num_samples]

    plt.figure(figsize=(20, 15))
    for i, idx in enumerate(sampled_indices):
        img_path = generator.filepaths[idx]
        img = Image.open(img_path)
        true_label = class_labels[true_classes[idx]]
        pred_label = class_labels[predictions[idx]]

        plt.subplot(6, 5, i + 1)
        plt.imshow(img)
        plt.title(f"True: {true_label}, Pred: {pred_label}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

display_predictions(test_generator, y_pred, test_generator.classes, test_generator.class_indices)

## Classification Report

In [ ]:
print('Classification Report')
target_names = list(test_generator.class_indices.keys())
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

## Matrice di confusione

In [ ]:
print('Confusion Matrix')
conf_matrix = confusion_matrix(test_generator.classes, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()